For this week, you will required to submit the following:

    A description of the problem and a discussion of the background. (15 marks)
    A description of the data and how it will be used to solve the problem. (15 marks)

#### Clearly define a problem or an idea of your choice, where you would need to leverage the Foursquare location data to solve or execute. Remember that data science problems always target an audience and are meant to help a group of stakeholders solve a problem, so make sure that you explicitly describe your audience and why they would care about your problem.
#### This submission will eventually become your Introduction/Business Problem section in your final report. So I recommend that you push the report (having your Introduction/Business Problem section only for now) to your Github repository and submit a link to it.

Quite often, when people eat out, they will usually prefer to have a certain type of food that are popular in that area.  For example, If I went to Japanese town in the city, I prefer to have Japanese food. Sometimes when I got there, there was a long queue. What I usually do is that I will drive to nearby Japanese restaurants that may have no or fewer waiting time.  In order to do so, I have to know which area has the most Japanese restaurants. If one is full I still have other choices. 

#### Describe the data that you will be using to solve the problem or execute your idea. Remember that you will need to use the Foursquare location data to solve the problem or execute your idea. You can absolutely use other datasets in combination with the Foursquare location data. So make sure that you provide adequate explanation and discussion, with examples, of the data that you will be using, even if it is only Foursquare location data.

#### This submission will eventually become your Data section in your final report. So I recommend that you push the report (having your Data section) to your Github repository and submit a link to it.

First, I need to choose the city I am interested.  Then I need to get all the restaurants information within the location (within 5000 m) using foursquare API. 
Then I need rank the high frequent type of food and decide which I want to have.  I will use the map visualization to help to make it easiser to see the results.

# ============= Coding part =====================

In [1]:
!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
!conda install -c conda-forge beautifulsoup4 --yes
!conda install -c conda-forge lxml --yes
print('packages installed.') 

Fetching package metadata .............
Solving package specifications: .

Package plan for installation in environment /opt/conda/envs/DSX-Python35:

The following NEW packages will be INSTALLED:

    altair:  2.2.2-py35_1 conda-forge
    branca:  0.3.1-py_0   conda-forge
    folium:  0.5.0-py_0   conda-forge
    vincent: 0.4.4-py_1   conda-forge

altair-2.2.2-p 100% |################################| Time: 0:00:00  42.36 MB/s
branca-0.3.1-p 100% |################################| Time: 0:00:00  36.54 MB/s
vincent-0.4.4- 100% |################################| Time: 0:00:00  40.49 MB/s
folium-0.5.0-p 100% |################################| Time: 0:00:00  48.49 MB/s
Fetching package metadata .............
Solving package specifications: .

Package plan for installation in environment /opt/conda/envs/DSX-Python35:

The following packages will be UPDATED:

    beautifulsoup4: 4.6.0-py35h442a8c9_1 --> 4.6.3-py35_0 conda-forge

beautifulsoup4 100% |################################| Time: 0

In [2]:
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files

#!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans


import folium # map rendering library

print('Libraries imported.')

Libraries imported.


In [41]:
CLIENT_ID = '051XT2EHQXLNFNGURXBLIXYMGCHPVV0ESEHPUSIQMO2JDZC0' # your Foursquare ID
CLIENT_SECRET = 'XGPSWEG1JGGHGYRAA4YRNWSRTWGB4KZLARQ2UOI0HB3IBHIS' # your Foursquare Secret
VERSION = '20190401'

In [127]:
address = ['San Jose, CA']


In [113]:
# geolocator = Nominatim(user_agent="foursquare_agent")
# latitude = []
# longitude =[]
# for x in address:
#     location = geolocator.geocode(x)
#     latitude.append(location.latitude)
#     longitude.append(location.longitude)
# print(latitude,longitude)

In [128]:
geolocator = Nominatim(user_agent="foursquare_agent")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print(latitude,longitude)

37.3361905 -121.8905833


In [32]:
# def getNearbyVenues(names, latitudes, longitudes, radius):
    
#     venues_list=[]
#     for name, lat, lng in zip(names, latitudes, longitudes):
#         print(name)
            
#         # create the API request URL
#         url = 'https://api.foursquare.com/v2/venues//search?&client_id={}&client_secret={}&v={}&ll={},{}}&query={}&radius={}&limit={}'.format(
#             CLIENT_ID, 
#             CLIENT_SECRET, 
#             VERSION, 
#             lat, 
#             lng,
#             search_query,
#             radius, 
#             LIMIT)
            
#         # make the GET request
#         results = requests.get(url).json()["response"]['groups'][0]['items']
        
#         # return only relevant information for each nearby venue
#         venues_list.append([(
#             name, 
#             lat, 
#             lng, 
#             v['venue']['name'], 
#             v['venue']['location']['lat'], 
#             v['venue']['location']['lng'],
#             v['venue']['categories'][0]['name']) for v in results])

#     nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
#     nearby_venues.columns = ['Neighborhood', 
#                   'Neighborhood Latitude', 
#                   'Neighborhood Longitude', 
#                   'Venue', 
#                   'Venue Latitude', 
#                   'Venue Longitude', 
#                   'Venue Category']
    
#     return(nearby_venues)

In [33]:
# radius = 5000
# LIMIT = 1000
# search_query = 'food'

# venues = getNearbyVenues(address, latitude, longitude, radius = 5000)



Palo Alto, CA


ValueError: Single '}' encountered in format string

In [129]:
radius = 10000
LIMIT = 5000
search_query = 'food'

In [130]:
url = 'https://api.foursquare.com/v2/venues/explore?client_id={}&client_secret={}&ll={},{}&v={}&section={}&radius={}&limit={}'.format(CLIENT_ID, CLIENT_SECRET, latitude, longitude, VERSION, search_query, radius, LIMIT)
url

'https://api.foursquare.com/v2/venues/explore?client_id=051XT2EHQXLNFNGURXBLIXYMGCHPVV0ESEHPUSIQMO2JDZC0&client_secret=XGPSWEG1JGGHGYRAA4YRNWSRTWGB4KZLARQ2UOI0HB3IBHIS&ll=37.3361905,-121.8905833&v=20190401&section=food&radius=10000&limit=5000'

In [132]:
results = requests.get(url).json()
# results['response']['groups'][0]['items']

In [133]:
# assign relevant part of JSON to venues
venues = results['response']['groups'][0]['items']

# tranform venues into a dataframe
dataframe = json_normalize(venues)

# filter columns
filtered_columns = ['venue.name', 'venue.categories'] + [col for col in dataframe.columns if col.startswith('venue.location.')] + ['venue.id']
dataframe_filtered = dataframe.loc[:, filtered_columns]

# filter the category for each row
dataframe_filtered['venue.categories'] = dataframe_filtered.apply(get_category_type, axis=1)

# clean columns
dataframe_filtered.columns = [col.split('.')[-1] for col in dataframe_filtered.columns]

dataframe_filtered


,name,categories,address,cc,city,country,crossStreet,distance,formattedAddress,labeledLatLngs,lat,lng,neighborhood,postalCode,state,id
0,San Pedro Square Market,Food Court,87 N San Pedro St,US,San Jose,United States,at W St John St,339,"[87 N San Pedro St (at W St John St), San Jose...","[{'label': 'display', 'lng': -121.894402639345...",37.336480,-121.894403,Downtown San Jose,95110,CA,4e6750db152001e1f7122a11
1,Back A Yard Caribbean American Grill,Caribbean Restaurant,80 N Market St,US,San Jose,United States,btwn St. John & Santa Clara,199,"[80 N Market St (btwn St. John & Santa Clara),...","[{'label': 'display', 'lng': -121.892749472995...",37.336683,-121.892749,NaN,95113,CA,4faabe59e4b092915bfa4229
2,Ike's Love And Sandwiches,Sandwich Place,75 E Santa Clara St Ste 130,US,San Jose,United States,at N 2nd St,125,"[75 E Santa Clara St Ste 130 (at N 2nd St), Sa...","[{'label': 'display', 'lng': -121.889436038090...",37.336852,-121.889436,NaN,95113,CA,5293dcf9498ee4056f1ecc87
3,Silicon Valley Capital Club,Restaurant,50 W San Fernando St,US,San Jose,United States,NaN,273,"[50 W San Fernando St, San Jose, CA 95113, Uni...","[{'label': 'display', 'lng': -121.8893173, 'la...",37.333945,-121.889317,NaN,95113,CA,4a6baaf9f964a5208dcf1fe3
4,Good Karma,Vegetarian / Vegan Restaurant,37 S 1st St,US,San Jose,United States,NaN,90,"[37 S 1st St, San Jose, CA 95113, United States]","[{'label': 'display', 'lng': -121.890093851189...",37.335476,-121.890094,NaN,95113,CA,4a55443af964a520f3b31fe3
5,Henry's Hi-Life,BBQ Joint,301 W Saint John St,US,San Jose,United States,near the Julian/St. James exit off CA87,669,[301 W Saint John St (near the Julian/St. Jame...,"[{'label': 'display', 'lng': -121.898079524972...",37.335405,-121.898080,NaN,95110,CA,4a53c067f964a520b2b21fe3
6,Teske's Germania,German Restaurant,255 N 1st St,US,San Jose,United States,at Devine St,469,"[255 N 1st St (at Devine St), San Jose, CA 951...","[{'label': 'display', 'lng': -121.893370169741...",37.339784,-121.893370,NaN,95113,CA,40d77680f964a52060011fe3
7,Freshly Baked Eatery,Sandwich Place,152 N 3rd St,US,San Jose,United States,btwn E St James & E St John St,360,[152 N 3rd St (btwn E St James & E St John St)...,"[{'label': 'display', 'lng': -121.890050924027...",37.339404,-121.890051,NaN,95112,CA,4abbbdd9f964a520a38420e3
8,Ludwig's German Table,German Restaurant,261 N 2nd St,US,San Jose,United States,NaN,503,"[261 N 2nd St, San Jose, CA 95112, United States]","[{'label': 'display', 'lng': -121.892617457880...",37.340414,-121.892617,NaN,95112,CA,575377b2498eeb535c579976
9,Morton's The Steakhouse,Steakhouse,177 Park Ave Ste 100,US,San Jose,United States,at S Market St.,479,"[177 Park Ave Ste 100 (at S Market St.), San J...","[{'label': 'display', 'lng': -121.8910747, 'la...",37.331902,-121.891075,NaN,95113,CA,4ad502fbf964a520450121e3


In [134]:
dataframe_filtered.shape

(100, 16)

In [151]:
rank = dataframe_filtered['categories'].value_counts().reset_index()

In [144]:
# from collections import Counter
# dict(Counter(dataframe_filtered['categories']))

In [168]:
rank

,index,categories
0,Mexican Restaurant,14
1,Sandwich Place,8
2,Pizza Place,8
3,Vietnamese Restaurant,5
4,Breakfast Spot,4
5,Ethiopian Restaurant,3
6,Korean Restaurant,3
7,Vegetarian / Vegan Restaurant,3
8,Café,3
9,Greek Restaurant,3


In [165]:
first = dataframe_filtered[dataframe_filtered['categories'] == rank['index'][0]]
second = dataframe_filtered[dataframe_filtered['categories'] == rank['index'][1]]
third = dataframe_filtered[dataframe_filtered['categories'] == rank['index'][2]]

In [167]:
venues_map = folium.Map(location=[latitude, longitude], zoom_start=12) # generate map centred around Ecco


# add Ecco as a red circle mark
folium.features.CircleMarker(
    [latitude, longitude],
    radius=10,
    popup="address",
    fill=True,
    color='red',
    fill_color='red',
    fill_opacity=0.6
    ).add_to(venues_map)


# add 1st frequent food to the map as blue circle markers
for lat, lng, label in zip(first.lat, first.lng, first.categories):
    folium.features.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        fill=True,
        color='blue',
        fill_color='blue',
        fill_opacity=0.6
        ).add_to(venues_map)
    
# add 2nd frequent food to the map as blue circle markers
for lat, lng, label in zip(second.lat, second.lng, second.categories):
    folium.features.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        fill=True,
        color='green',
        fill_color='green',
        fill_opacity=0.6
        ).add_to(venues_map)
    
# add 3rd frequent food to the map as blue circle markers
for lat, lng, label in zip(third.lat, third.lng, third.categories):
    folium.features.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        fill=True,
        color='yellow',
        fill_color='yellow',
        fill_opacity=0.6
        ).add_to(venues_map)

# display map
venues_map